# Summary

To prepare for our first lab, let's develop some familiarity with
`CVXR`. It's the `R` package we'll be using to formulate and solve
optimization problems that come up throughout the semester. In this
homework, we'll use it for linear regression. That is, we'll solve this
optimization problem.

$$
\hat\mu(x) = \hat\beta_0 + \hat\beta_1 x \quad \text{ where } \quad \hat \beta = \operatorname*{argmin}_{b \in \mathbb{R}^2} \frac{1}{n}\sum_{i=1}^n \{ Y_i - (b_0 + b_1 X_i) \}^2.
$$

We could, of course, just use `lm` for this, but we're going to be
solving some other problems `lm` can't handle and we can use the
practice. `CVXR` lets us write down convex optimization problems like
the one above in a special subset of ordinary `R` code. It chooses an
appropriate numerical solver for us, translates our input into what that
solver expects, runs it, and gives us the result. So in essence, if you
write down an expression to optimize and a list of constraints in
language that looks more or less like math, it gives you the solution.

We'll use a few libraries: `CVXR` for optimization, `ggplot2` for plotting.

In [ ]:
suppressPackageStartupMessages({
    library(ggplot2)
    library(CVXR)
})

# OSQP claims some feasible problems aren't, so we'll tell CVXR not to use it
CVXR::add_to_solver_blacklist('OSQP')  

# And we'll style our plots  
theme_update(plot.background = element_rect(fill = "transparent", colour = NA),
		    panel.background = element_rect(fill = "transparent", colour = NA),
                    legend.background = element_rect(fill="transparent", colour = NA),
                    legend.box.background = element_rect(fill="transparent", colour = NA),
                    legend.key = element_rect(fill="transparent", colour = NA),
			panel.grid.major=element_line(color=rgb(1,0,0,.1,  maxColorValue=1)),
	        panel.grid.minor=element_line(color=rgb(0,0,1,.1,  maxColorValue=1)),
		    axis.ticks.x = element_blank(),
		    axis.ticks.y = element_blank(),
		    axis.text.x  = element_text(colour = "#aaaaaa"),
		    axis.text.y  = element_text(colour = "#aaaaaa"),
		    axis.title.x  = element_text(colour = "#aaaaaa"),
		    axis.title.y  = element_text(colour = "#aaaaaa", angle=90))

# Fitting our curve

We'll be using `CVXR` a lot, so you'll get used to it. To get a sense 
of how it works, let's take a look at the `CVXR` code for fitting a line
to a set of points $(X_1, Y_1) \ldots (X_n, Y_n)$. 


There are basically four things we need to do in this code.

1. 
  - a. We specify the variable(s) we're optimizing over. This is a vector $b \in \mathbb{R}^2$,
  so we'll tell `CVXR` we're working with a vector of two variables called `b`.  
  - b. What we're interested in are the set of predictions 
    $m(x) = b_0 + b_1 x$ corresponding to those variables. So we'll 
    write, in `R` terms, an expression for the vector of predictions 
    $m(X_1) \ldots m(X_n)$ in terms of those two variables.

2.  We specify, in terms of those predictions, what we want to minimize.
    That's mean squared error,
    $\frac{1}{n}\sum_{i=1}^n \{ Y_i - m(X_i) \}^2$.

3.  We ask CVXR to actually minimize it and to tell us at which values
    of $b$ this minimum occurs.

4.  We organize the results into a structure that'll be convenient
    later: a list with our results.

In [ ]:
linereg = function(X,Y) {  
  # Here's Step 1a.
  # We tell CVXR we're thinking about a vector b in R^2
  b = Variable(2)
  # Here's Step 1b.
  # We tell CVXR how to compute the predictions m(X) in terms of b.
  m = b[1] + b[2]*X

  # Here's Step 2.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - m)^2) / n

  # Here's Step 3.
  # We ask CVXR to minimize mean squared error and ask for vector b that does it.
  solved = solve(Problem(Minimize(mse))) 
  beta.hat = solved$getValue(b)
  mu.hat = solved$getValue(m)
  
  # Here's Step 4: a little boilerplate to make it idiomatic R. 
  #  1. we record beta.hat and corresponding predictions mu.hat in a list. We also record the input data.
  #  2. we assign that list a class, so R knows predict should delegate to predict.linereg
  #  3. we return the list
  model = list(beta.hat=beta.hat, mu.hat=mu.hat, input=list(X=X,Y=Y))
  attr(model, "class") = "linereg"
  model
}

## A Note about Object Oriented Programming in R

Object Oriented Programming in R is, at a basic level, very easy. If we
have a list `that.list`, and we've set its 'class' attribute to
`linearreg`, then when we call `predict(that.list, ...)`, `R` will
actually call `predict.linearreg(that.list, ...)`. This is useful
because it means we don't need to change code that makes and uses
predictions when we change our model.



# Trying it out

Now let's fit some data and plot a curve through it. The data we'll be
looking at is fake and stylized. We'll look at points sampled around a
'stepline' function that jumps from 0 to the line $y=x$ at $x=0.5$. 

$$
\mu(x) = \begin{cases} 0 & \quad \text{if} \quad x < .5 \\ 
                       x & \quad \text{if} \quad x \ge .5 \end{cases}
$$ 

Let's start by plotting the data.


In [ ]:
mu = function(x) { x*(x >= .5) }
sigma = .1

n = 100
X = runif(n)
Y = mu(X) + sigma*rnorm(n)

observations = ggplot() + geom_point(aes(x=X, y=Y), alpha=.2)
observations

Now let's use `linereg` to fit a line $\hat\mu(x)=\hat\beta_0 + \hat\beta_1 x$ and take a look at the predictions $\hat\mu(X_i)$ of our least squares line at the observed points $X_i$.

In [ ]:
model = linereg(X,Y)
mu.hat = model$mu.hat
observations + geom_point(aes(x=X, y=mu.hat), color='blue', alpha=.2)

# Making predictions between the observations

Let's implement `predict.linearreg`. `R` convention expects a predict
function like this to take two arguments.

1. This list we've been
talking about, which describes the curve we've fit.
2. A data frame describing the points $X_i$ that we want to make predictions at. 

Conventionally that the second argument is optional, and if it's not passed,
the function should make predictions at the the points $X_i$ you used to fit the model. 

Take a look at this implementation. This is the function that gets called when we say
`predict(model)` or `predict(model, newdata=data.frame(x=...))` for a model with class `'linereg'`.

In [ ]:
predict.linereg = function(model, newdata=data.frame(X=model$input$X)) {
    X=newdata$X 
    beta.hat = model$beta.hat
    beta.hat[1] + X*beta.hat[2]
}

Easy, right? We're making predictions
$\hat\mu(x) = \hat\beta_0 + \hat\beta_1 X$ for the points $X$ in the
data frame `newdata`. And if you don't pass `newdata` we just use the
points $X$ from our training data, which we saved when we wrote
`linearreg` just for this purpose.

Now let's fit the data and plot a curve through it. To plot this curve,
we'll make predictions at a points on a dense sequence of Xs and ask `R`
to draw a line through those predictions.

In [ ]:
model = linereg(X,Y)

x = seq(0,1,by=.001)
muhat.x = predict(model, newdata=data.frame(X=x))
observations + geom_line(aes(x=x, y=muhat.x), color='blue')

Now let's check that we've done things right by using the built-in
function `lm` to do the same. This is a one line change.

In [ ]:
model = lm(Y~X)

x = seq(0,1,by=.001)
muhat.x = predict(model, newdata=data.frame(X=x))
observations + geom_line(aes(x=x, y=muhat.x), color='blue') 

# Constrained Linear Regression

Sometimes we have domain knowledge we want to use when we fit our line. 
For example, we might think that our slope $b_1$ should be non-negative. 
Like if we're predicting height in childhood as a function of age: kids grow, but they
don't shrink. While it might not be necessary in a simple 2-coefficient
model, in more complex models imposing constraints based on domain
knowledge tends to improve performance. 

To encode this knowledge, we add a step to what we've done above.

 - Step 2a. We specify constraints on the coefficients beta that our
solution must satisfy.

In [ ]:
increasinglinereg = function(X,Y) {  
  # Here's Step 1. Same as before.
  # 1a. We tell CVXR we're thinking about a vector b in R^2
  b = Variable(2)
  # 1b. We tell CVXR how to compute the predictions m(X) in terms of b.
  m = b[1] + b[2]*X
  
  # Here's Step 2. Same as before.
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - m)^2) / n

  # Here's Step 2a. This is new.
  # We specify our constraints.
  nonneg.constraint = b[2] >= 0
  
  # Here's Step 3. This changes to incorporate the constraint. 
  # We ask CVXR to minimize mean squared error, subject to our constraints,
  # and ask for vector b that does it. Note that because CVXR allows us to
  # use multiple constraints, it expects us to pass a list of constraints;
  # because we have only one here, we pass it a one-element list.
  solved = solve(Problem(Minimize(mse), list(nonneg.constraint)))
  beta.hat = solved$getValue(b)
  mu.hat = solved$getValue(m)
  
  # Here's Step 4: a little boilerplate to make it idiomatic R. Same as before.
  #  1. we record the input X and the solution beta.hat and corresponding predictions mu.hat in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.increasinglinereg
  #  3. we return the list
  model = list(beta.hat=beta.hat, mu.hat=mu.hat, input=list(X=X,Y=Y))
  attr(model, "class") = "increasinglinereg"
  model
}

Note that while we've changed the way we solve for our coefficients, we're making predictions just as before:
$\hat \mu(x) = \hat\beta_0 + \hat\beta_1 x$. That means we can use the prediction function we wrote above.

In [ ]:
predict.increasinglinereg = predict.linereg

And we can make and plot our predictions just as before.

In [ ]:
model = increasinglinereg(X,Y)

x = seq(0,1,by=.001)
muhat.x = predict(model, newdata=data.frame(X=x))
observations + geom_line(aes(x=x, y=muhat.x), color='blue') 

Nothing has changed because our data follows an increasing trend: we get
$\hat\beta_1 \ge 0$ even without the constraint. But let's check that
something will change if our data isn't increasing. We'll look at points
sampled around a negated version of our 'stepline'. 

$$
\mu(x) = \begin{cases} \hphantom{-}0 & \quad \text{if} \quad x < .5 \\ 
                       -x & \quad \text{if} \quad x \ge .5 \end{cases}
$$ 

Here's the new data.

In [ ]:
mu = function(x) { -x*(x >= .5) }
sigma = .1

n = 100
X = runif(n)
Y = mu(X) + sigma*rnorm(n)

observations = ggplot() + geom_point(aes(x=X, y=Y), alpha=.2)
observations

And here are fits using `linereg` (the unconstrained regression) in
blue and `increasinglinereg` (the increasingness-constrained one) in red. The
increasingness-constrained regression can't fit the data, so it does the
best it can. It gives us a horizontal line.

In [ ]:
model.uc = linereg(X,Y)
model.nn = increasinglinereg(X,Y)

x = seq(0,1,by=.001)
muhat.x.uc = predict(model.uc, newdata=data.frame(X=x))
muhat.x.nn = predict(model.nn, newdata=data.frame(X=x))

observations + geom_line(aes(x=x, y=muhat.x.uc), color='blue') +
               geom_line(aes(x=x, y=muhat.x.nn), color='red')


## Doing it yourself

Try implementing a version of linear regression in which the coefficient
$\beta_1$ is constrained to be *small* in magnitude. Given some bound
$B$ on how large it can be, we'll solve this problem.

$$
\hat\mu(x) = \hat\beta_0 + \hat\beta_1 x \quad \text{ where } \quad
\hat \beta = \operatorname*{argmin}_{\substack{b \in \mathbb{R}^2 \\ \lvert b_1\rvert \le B}} \frac{1}{n}\sum_{i=1}^n \{ Y_i - (b_0 + b_1 X_i) \}^2.
$$

Give it a shot. Here's a template to work with. 

**Tip**. `CVXR` knows what functions like `abs` mean, so you can use them in your constraints.

In [ ]:
boundedlinereg = function(X,Y,B) {  
  # Step 1. 
  # We tell CVXR we're thinking about a vector b in R^2
 

  # Step 2. 
  # We tell CVXR that we're interested in mean squared error.

  # Step 2a. 
  # We specify our constraints.
  
  # Step 3. 
  # We ask CVXR to minimize mean squared error, subject to our constraints,
  # and ask for vector b that does it. 
 
  # Here's Step 4: a little boilerplate to make it idiomatic R. Same as before.
  #  1. we record the input X and the solution beta.hat in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.boundedlinereg
  #  3. we return the list
  model = list(beta.hat=beta.hat, input=list(X=X,Y=Y,B=B))
  attr(model, "class") = "boundedlinereg"
  model
}

predict.boundedlinereg = predict.linereg

In [ ]:
boundedlinereg = function(X,Y,B) {  
  # Step 1. 
  # We tell CVXR we're thinking about a vector b in R^2
  b = Variable(2) 

  # Step 2. 
  # We tell CVXR that we're interested in mean squared error.
  mse = sum((Y - (b[1] + b[2]*X))^2) / n

  # Step 2a. 
  # We specify our constraints.
  boundedness.constraint = abs(b[2]) <= B

  # Step 3. 
  # We ask CVXR to minimize mean squared error, subject to our constraints,
  # and ask for vector b that does it. 
  solved = solve(Problem(Minimize(mse), list(boundedness.constraint))) 
  beta.hat = solved$getValue(b)

  # Here's Step 4: a little boilerplate to make it idiomatic R. Same as before.
  #  1. we record the input X and the solution beta.hat in a list
  #  2. we assign that list a class, so R knows predict should delegate to predict.boundedlinereg
  #  3. we return the list
  model = list(beta.hat=beta.hat, input=list(X=X,Y=Y,B=B))
  attr(model, "class") = "boundedlinereg"
  model
}

predict.boundedlinereg = predict.linereg

Now we'll use your code to make predictions for two values of the bound
$B$: a big one ($B=1.0$ in blue) and a small one ($B=0.1$ in red).

In [ ]:
model.big   = boundedlinereg(X,Y, B=1.0)
model.small = boundedlinereg(X,Y, B=0.1)

x = seq(0,1,by=.001)
muhat.x.big   = predict(model.big,   newdata=data.frame(X=x))
muhat.x.small = predict(model.small, newdata=data.frame(X=x))

observations + geom_line(aes(x=x, y=muhat.x.big),   color='blue') +
               geom_line(aes(x=x, y=muhat.x.small), color='red')

Note that while we do a better job with the larger value of the bound,
we're still not doing a great job. After all, the curve $\mu(x)$ our
data is sampled around isn't a line. So let's conclude for now. 

In lab, we'll start working on fitting *monotone* curves, i.e. curves that are increasing or decreasing, and we'll do better.